In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
import torch
from torchvision import transforms, datasets, models
import torchvision as tv
import torch.nn as nn


model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features
print("num_ftrs: ", num_ftrs)

class FineTuneModel(nn.Module):
    def __init__(self, original_model, num_filters, num_classes):
        super().__init__()
        self.body = nn.Sequential(*(list(original_model.children())[:-1]))
        self.head = nn.Sequential(
                        nn.BatchNorm1d(num_filters),
                        nn.Dropout(0.5),
                        nn.Linear(num_filters, 512),
                        nn.ReLU(True),
                        nn.BatchNorm1d(512),
                        nn.Dropout(0.5),
                        nn.Linear(512, 120),
                    )

    def forward(self, x):
        f = self.body(x)
        # f = torch.flatten(f)
        f = f.view(f.size(0), -1)
        y = F.log_softmax(self.head(f))
        return y


model_ft = FineTuneModel(model_ft, num_ftrs, 120)
model_ft.load_state_dict(torch.load('final_breed_clf.pth'))

num_ftrs:  2048


<All keys matched successfully>

In [2]:
import pandas as pd
import numpy as np
labels = pd.read_csv('labels.csv')
# sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
labels

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


In [3]:


labels['target'] = 1
labels_pivot = labels.pivot(index = 'id',columns =  'breed',values =  'target').reset_index().fillna(0)

classes = labels_pivot.columns[1:]
classes

Index(['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale',
       'american_staffordshire_terrier', 'appenzeller', 'australian_terrier',
       'basenji', 'basset', 'beagle',
       ...
       'toy_poodle', 'toy_terrier', 'vizsla', 'walker_hound', 'weimaraner',
       'welsh_springer_spaniel', 'west_highland_white_terrier', 'whippet',
       'wire-haired_fox_terrier', 'yorkshire_terrier'],
      dtype='object', name='breed', length=120)

In [4]:
classes[19]

'boston_bull'

In [5]:
import torch.nn.functional as F

top=tk.Tk()
top.geometry('800x600')
top.title('Dog Breed classification')
top.configure(background='#51d3bf')
label=Label(top,background='#FFFFFF', font=('arial',15,'bold'))
breed_image = Label(top)

def classify(file_path,model):
    global label_packed
    image = Image.open(file_path)
    model.cuda()
    data_transforms = transforms.Compose([
    transforms.Resize((448,448)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
    ])
    
    image = data_transforms(image)
    image.unsqueeze_(dim=0)
    model.eval()
    with torch.no_grad():
        pred = model(image.cuda())
        _,preds = torch.max(pred, dim=1)
        print(preds)
    
    breed = classes[preds]
    label.configure(foreground='#011638', text="This dog is of "+breed+" Breed") 
    
def show_classify_button(file_path,model):
    classify_b=Button(top,text="Classify Image",command=lambda: classify(file_path,model),padx=10,pady=5)
    classify_b.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)
    
def upload_image():
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        breed_image.configure(image=im)
        breed_image.image=im
        label.configure(text='')
        show_classify_button(file_path,model_ft)
    
    
upload=Button(top,text="Upload an image",command=upload_image,padx=10,pady=5)

upload.configure(background='#e51369', foreground='white',font=('arial',10,'bold'))

upload.pack(side=BOTTOM,pady=50)
breed_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)

heading = Label(top, text="Know your dog breed",pady=20, font=('arial',20,'bold'))
heading.configure(background='#ea1236',foreground='#364156')
heading.pack()
top.mainloop()

C:\Users\wwech\Anaconda3\envs\env_pytorch\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([16], device='cuda:0')
tensor([33], device='cuda:0')
